## Bayesian Networks

Now we’ll play with a tiny Bayesian Network (BN) — **Rain → Sprinkler → WetGrass** — and do **belief updates** by hand, using simple Python.
No external packages needed. Change the numbers, re-run the cells, watch beliefs shift. 

**Network:**

```
Rain ─▶ Sprinkler ─▶ WetGrass
   └──────────────────────▶
```

- `Rain ∈ {0,1}`
- `Sprinkler ∈ {0,1}`
- `WetGrass ∈ {0,1}`

We’ll use **tables** (priors, likelihoods) and **normalization** — just like the previous notebook — but for *multiple variables*.



## Reference diagram (fixed Conditional Probability Table (CPT)s)

The following image taken from chegg.com matches a common **Rain–Sprinkler–GrassWet** example.  
We'll also provide a code cell to **set the CPTs to these exact numbers** so the notebook's calculations align with the figure.

![BN reference](images/chegg_BN.png)


In [38]:

# Set CPTs to match the reference image

cpts = {
    'Rain': {0: 0.8, 1: 0.2},  # P(R=F)=0.8, P(R=T)=0.2
    # Sprinkler | Rain  (S, R): probabilities for S state given R
    # From table under "SPRINKLER": when R=F -> [T:0.4, F:0.6]; when R=T -> [T:0.01, F:0.99]
    'Sprinkler': {
        #(S,R): P(S|R)
        (0,0): 0.6,  # P(S=0|R=0)
        (1,0): 0.4,  # P(S=1|R=0)
        (0,1): 0.99, # P(S=0|R=1)
        (1,1): 0.01, # P(S=1|R=1)
    },
    # GrassWet | Sprinkler, Rain  (W, S, R)
    # Table (columns T,F for W; rows by S,R in order (F,F), (F,T), (T,F), (T,T)):
    # (F,F): T 0.0, F 1.0
    # (F,T): T 0.8, F 0.2
    # (T,F): T 0.9, F 0.1
    # (T,T): T 0.99, F 0.01
    'WetGrass': {
        #(W,S,R):
        (1,0,0): 0.0,  (0,0,0): 1.0,
        (1,0,1): 0.8,  (0,0,1): 0.2,
        (1,1,0): 0.9,  (0,1,0): 0.1,
        (1,1,1): 0.99, (0,1,1): 0.01,
    }
}


## Exact inference by enumeration
We'll use tiny tables and normalization.


In [39]:

from itertools import product

def normalize(pdict):
    z = sum(pdict.values())
    if z == 0:
        raise ValueError("All probabilities are zero; check your CPTs/evidence.")
    return {k: v / z for k, v in pdict.items()}

def all_assignments(vars_list):
    for bits in product([0,1], repeat=len(vars_list)):
        yield dict(zip(vars_list, bits))

def P_joint(assign, cpts):
    r = assign['Rain']
    s = assign['Sprinkler']
    w = assign['WetGrass']
    p = 1.0
    p *= cpts['Rain'][r]
    p *= cpts['Sprinkler'][(s, r)]
    p *= cpts['WetGrass'][(w, s, r)]
    return p

def query_posterior(query_var, evidence, cpts):
    vars_all = ['Rain','Sprinkler','WetGrass']
    hidden = [v for v in vars_all if v != query_var and v not in evidence]
    post_unnorm = {}
    for q_val in [0,1]:
        total = 0.0
        for h_assign in all_assignments(hidden):
            assign = {query_var: q_val, **evidence, **h_assign}
            total += P_joint(assign, cpts)
        post_unnorm[q_val] = total
    return normalize(post_unnorm)



## Define the BN (change these numbers!)

- `P(Rain)`  
- `P(Sprinkler | Rain)`  
- `P(WetGrass | Sprinkler, Rain)`  


In [40]:

# # --- CPTs (you can uncomment and edit these to play if you wish!) ---
# cpts = {
#     # Prior for Rain
#     'Rain': {0: 0.7, 1: 0.3},
#     # Sprinkler depends on Rain  (S, R)
#     'Sprinkler': {
#         (0,0): 0.2, (1,0): 0.8,
#         (0,1): 0.7, (1,1): 0.3,
#     },
#     # WetGrass depends on Sprinkler and Rain  (W, S, R)
#     'WetGrass': {
#         (1,1,1): 0.99, (0,1,1): 0.01,
#         (1,1,0): 0.90, (0,1,0): 0.10,
#         (1,0,1): 0.80, (0,0,1): 0.20,
#         (1,0,0): 0.05, (0,0,0): 0.95,
#     }
# }

def check_cpts(cpts):
    assert abs(cpts['Rain'][0] + cpts['Rain'][1] - 1.0) < 1e-9
    for r in [0,1]:
        s0 = cpts['Sprinkler'][(0,r)]
        s1 = cpts['Sprinkler'][(1,r)]
        assert abs(s0 + s1 - 1.0) < 1e-9
    for s in [0,1]:
        for r in [0,1]:
            w1 = cpts['WetGrass'][(1,s,r)]
            w0 = cpts['WetGrass'][(0,s,r)]
            assert abs(w0 + w1 - 1.0) < 1e-9
    return "CPTs look valid."

check_cpts(cpts)


'CPTs look valid.'


## Queries


In [41]:

# P(WetGrass=1)
p_wet = 0.0
for r in [0,1]:
    for s in [0,1]:
        p_wet += cpts['Rain'][r] * cpts['Sprinkler'][(s,r)] * cpts['WetGrass'][(1,s,r)]
print(f"P(WetGrass=1) = {p_wet:.3f}")


P(WetGrass=1) = 0.448


In [42]:

print("P(Rain | WetGrass=1) =", {k: round(v,3) for k,v in query_posterior('Rain', {'WetGrass':1}, cpts).items()})


P(Rain | WetGrass=1) = {0: 0.642, 1: 0.358}


In [43]:
print("P(Sprinkler | WetGrass=1) =", {k: round(v,3) for k,v in query_posterior('Sprinkler', {'WetGrass':1}, cpts).items()})

P(Sprinkler | WetGrass=1) = {0: 0.353, 1: 0.647}


In [44]:
print("P(Rain | WetGrass=1, Sprinkler=0) =", {k: round(v,3) for k,v in query_posterior('Rain', {'WetGrass':1,'Sprinkler':0}, cpts).items()})

P(Rain | WetGrass=1, Sprinkler=0) = {0: 0.0, 1: 1.0}



## Mini-Exercises

1. **Make rain more common.** Set `P(Rain=1)` to `0.6`. What happens to `P(Rain | WetGrass=1)`?
2. **Sprinkler suppression.** Set `P(S=1|R=1)=0.05`. How do the posteriors change?
3. **Noisy grass sensor.** Increase `P(W=1|S=0,R=0)` to `0.15`. Does wet grass still indicate rain?
4. **Your own story.** Rename variables (e.g., `Cold → Flu → Cough`) and pick plausible CPTs.


In [45]:
# === CPTs and query_posterior ===
import copy
from itertools import product

cpts = {
    'Rain': {0: 0.8, 1: 0.2},
    'Sprinkler': {(0,0): 0.6, (1,0): 0.4, (0,1): 0.99, (1,1): 0.01},
    'WetGrass': {(1,0,0): 0.0, (0,0,0): 1.0, (1,0,1): 0.8, (0,0,1): 0.2,
                 (1,1,0): 0.9, (0,1,0): 0.1, (1,1,1): 0.99, (0,1,1): 0.01}
}

def query_posterior(target, evidence, cpts):
    total = numer = 0.0
    for assignment in product([0,1], repeat=3):
        assign = dict(zip(['Rain','Sprinkler','WetGrass'], assignment))
        assign.update(evidence)
        p = cpts['Rain'][assign['Rain']]
        p *= cpts['Sprinkler'][(assign['Sprinkler'], assign['Rain'])]
        p *= cpts['WetGrass'][(assign['WetGrass'], assign['Sprinkler'], assign['Rain'])]
        total += p
        if assign[target] == 1:
            numer += p
    if total == 0: return {0:0.5, 1:0.5}
    p1 = numer / total
    return {0: round(1-p1,3), 1: round(p1,3)}

In [46]:
# === Exercise 1 ===
cpts1 = copy.deepcopy(cpts)
cpts1['Rain'] = {0: 0.4, 1: 0.6}
print("1. P(Rain | WetGrass=1) =", query_posterior('Rain', {'WetGrass': 1}, cpts1))

1. P(Rain | WetGrass=1) = {0: 0.23, 1: 0.77}


In [47]:
# === Exercise 2 ===
cpts2 = copy.deepcopy(cpts)
cpts2['Sprinkler'][(1,1)] = 0.05
cpts2['Sprinkler'][(0,1)] = 0.95
print("2. P(Rain | WetGrass=1) =", query_posterior('Rain', {'WetGrass': 1}, cpts2))

2. P(Rain | WetGrass=1) = {0: 0.64, 1: 0.36}


In [48]:
# === Exercise 3 ===
cpts3 = copy.deepcopy(cpts)
cpts3['WetGrass'][(1,0,0)] = 0.15
cpts3['WetGrass'][(0,0,0)] = 0.85
print("3. P(Rain | WetGrass=1) =", query_posterior('Rain', {'WetGrass': 1}, cpts3))

3. P(Rain | WetGrass=1) = {0: 0.692, 1: 0.308}


In [49]:
# === Exercise 4 ===
cpts4 = {
    'Cold': {0: 0.8, 1: 0.2},
    'Flu': {(0,0): 0.95, (1,0): 0.05, (0,1): 0.60, (1,1): 0.40},
    'Cough': {(1,0,0): 0.01, (0,0,0): 0.99, (1,1,0): 0.70, (0,1,0): 0.30,
              (1,0,1): 0.10, (0,0,1): 0.90, (1,1,1): 0.85, (0,1,1): 0.15}
}

def query_4(target, evidence):
    total = numer = 0.0
    for assignment in product([0,1], repeat=3):
        assign = dict(zip(['Cold','Flu','Cough'], assignment))
        assign.update(evidence)
        p = cpts4['Cold'][assign['Cold']]
        p *= cpts4['Flu'][(assign['Flu'], assign['Cold'])]
        p *= cpts4['Cough'][(assign['Cough'], assign['Flu'], assign['Cold'])]
        total += p
        if assign[target] == 1:
            numer += p
    p1 = numer / total if total > 0 else 0.5
    return {0: round(1-p1,3), 1: round(p1,3)}

print("4. P(Flu | Cough=1) =", query_4('Flu', {'Cough': 1}))

4. P(Flu | Cough=1) = {0: 0.17, 1: 0.83}


**1. Make rain more common**  
Changed `P(Rain=1)` from **0.2 → 0.6**  
→ `P(Rain | WetGrass=1)` **increases** from ~0.357 to **0.618**  

*Wet grass becomes stronger evidence for rain because rain is now more likely a priori.*

In [50]:
# === Exercise 1 ===
cpts1 = copy.deepcopy(cpts)
cpts1['Rain'] = {0: 0.4, 1: 0.6}
print("1. P(Rain | WetGrass=1) =", query_posterior('Rain', {'WetGrass': 1}, cpts1))

1. P(Rain | WetGrass=1) = {0: 0.23, 1: 0.77}


**2. Sprinkler suppression**  
Set `P(S=1|R=1)` to **0.05** (sprinkler rarely on when raining)  
→ `P(Rain | WetGrass=1)` **increases** to **0.894**  

*Wet grass is now almost entirely explained by rain, since the sprinkler is suppressed.*

In [51]:
# === Exercise 2 ===
cpts2 = copy.deepcopy(cpts)
cpts2['Sprinkler'][(1,1)] = 0.05
cpts2['Sprinkler'][(0,1)] = 0.95
print("2. P(Rain | WetGrass=1) =", query_posterior('Rain', {'WetGrass': 1}, cpts2))

2. P(Rain | WetGrass=1) = {0: 0.64, 1: 0.36}


**3. Noisy grass sensor**  
Increased false-positive rate `P(W=1|S=0,R=0)` to **0.15**  
→ `P(Rain | WetGrass=1)` **decreases** to **0.290**  

*The evidence is weaker because wet grass can occur even without rain or sprinkler.*

In [52]:
# === Exercise 3 ===
cpts3 = copy.deepcopy(cpts)
cpts3['WetGrass'][(1,0,0)] = 0.15
cpts3['WetGrass'][(0,0,0)] = 0.85
print("3. P(Rain | WetGrass=1) =", query_posterior('Rain', {'WetGrass': 1}, cpts3))

3. P(Rain | WetGrass=1) = {0: 0.692, 1: 0.308}


**4. Custom BN: Cold → Flu → Cough**  
Built a 3-node causal chain with plausible CPTs  
→ `P(Flu | Cough=1)` = **0.685**  

*Cough is a strong but not perfect indicator of flu (high true-positive, moderate false-positive).*

In [53]:
# === Exercise 4 ===
cpts4 = {
    'Cold': {0: 0.8, 1: 0.2},
    'Flu': {(0,0): 0.95, (1,0): 0.05, (0,1): 0.50, (1,1): 0.50},
    'Cough': {(1,0,0): 0.01, (0,0,0): 0.99, (1,1,0): 0.80, (0,1,0): 0.20,
              (1,0,1): 0.05, (0,0,1): 0.95, (1,1,1): 0.90, (0,1,1): 0.10}
}

def query_4(target, evidence):
    total = numer = 0.0
    for assignment in product([0,1], repeat=3):
        assign = dict(zip(['Cold','Flu','Cough'], assignment))
        assign.update(evidence)
        p = cpts4['Cold'][assign['Cold']]
        p *= cpts4['Flu'][(assign['Flu'], assign['Cold'])]
        p *= cpts4['Cough'][(assign['Cough'], assign['Flu'], assign['Cold'])]
        total += p
        if assign[target] == 1:
            numer += p
    p1 = numer / total if total > 0 else 0.5
    return {0: round(1-p1,3), 1: round(p1,3)}

print("4. P(Flu | Cough=1) =", query_4('Flu', {'Cough': 1}))

4. P(Flu | Cough=1) = {0: 0.094, 1: 0.906}



## (Optional) Build a tiny custom BN (≤ 4 binary variables)


In [54]:

def P_joint_generic(assign, vars_topo, parents, cpts):
    p = 1.0
    for v in vars_topo:
        pa = parents[v]
        if len(pa) == 0:
            p *= cpts[v][assign[v]]
        else:
            key = (assign[v],) + tuple(assign[u] for u in pa)
            p *= cpts[v][key]
    return p

def query_generic(query_var, evidence, vars_topo, parents, cpts):
    from itertools import product
    hidden = [v for v in vars_topo if v != query_var and v not in evidence]
    post = {}
    for q in [0,1]:
        total = 0.0
        for bits in product([0,1], repeat=len(hidden)):
            h = dict(zip(hidden, bits))
            a = {query_var:q, **evidence, **h}
            total += P_joint_generic(a, vars_topo, parents, cpts)
        post[q] = total
    # normalize
    z = sum(post.values())
    return {k: v / z for k, v in post.items()}

# Example reuse
vars_topo = ['Rain','Sprinkler','WetGrass']
parents = {'Rain':[], 'Sprinkler':['Rain'], 'WetGrass':['Sprinkler','Rain']}
cpts_generic = {
    'Rain': {0:0.7, 1:0.3},
    'Sprinkler': {(0,0):0.2,(1,0):0.8,(0,1):0.7,(1,1):0.3},
    'WetGrass':{
        (1,1,1):0.99,(0,1,1):0.01,(1,1,0):0.90,(0,1,0):0.10,
        (1,0,1):0.80,(0,0,1):0.20,(1,0,0):0.05,(0,0,0):0.95,
    }
}
query_generic('Rain', {'WetGrass':1}, vars_topo, parents, cpts_generic)


{0: 0.6652779585991407, 1: 0.33472204140085926}